In [1]:
### Importing Libraries

import sys
print(sys.version)
import os
import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt
WORKSPACE_PATH  = os.environ['WORKSPACE_PATH']
plt.style.use(WORKSPACE_PATH+'/ROMNet/romnet/extra/postprocessing/presentation.mplstyle')

from scipy.integrate import solve_ivp

pd.options.mode.chained_assignment = None  # default='warn'

3.8.11 (default, Jun 29 2021, 03:17:05) 
[Clang 12.0.0 (clang-1200.0.32.29)]


In [2]:
### Input Data

OutputDir  = WORKSPACE_PATH+'/ROMNet/Data/0DReact_Isobaric_1Case/'
# TestDir    = WORKSPACE_PATH+'/ROMNet/Data/0DReact_Isobaric_100Cases_Test/'
try:
    os.makedirs(OutputDir)
except OSError as e:
    pass
# try:
#     os.makedirs(TestDir)
# except OSError as e:
#     pass

NShifts  = 20

In [5]:
FileName           = OutputDir+'/Orig/train/ext/y.csv.'+str(1)
DataRead           = pd.read_csv(FileName, header=0)

In [6]:
DataRead

,t,T,H2,H,O,O2,OH,H2O,HO2,H2O2,...,HCNO,HOCN,HNCO,NCO,N2,AR,C3H7,C3H8,CH2CHO,CH3CHO
0,0.000000e+00,1000.000000,0.027227,0.000000e+00,0.000000e+00,0.216077,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.0,0.0,0.0,0.756695,0.0,0.0,0.0,0.0,0.0
1,1.000000e-12,1000.000000,0.027227,1.368146e-18,6.058194e-19,0.216077,3.539582e-19,7.368854e-19,4.922018e-17,4.886984e-21,...,0.0,0.0,0.0,0.0,0.756695,0.0,0.0,0.0,0.0,0.0
2,2.636651e-12,1000.000000,0.027227,3.607323e-18,1.597334e-18,0.216077,9.332641e-19,1.942910e-18,1.297764e-16,1.288527e-20,...,0.0,0.0,0.0,0.0,0.756695,0.0,0.0,0.0,0.0,0.0
3,6.951928e-12,1000.000000,0.027227,9.511253e-18,4.211613e-18,0.216077,2.460692e-18,5.122774e-18,3.421751e-16,3.397396e-20,...,0.0,0.0,0.0,0.0,0.756695,0.0,0.0,0.0,0.0,0.0
4,1.832981e-11,1000.000000,0.027227,2.507785e-17,1.110455e-17,0.216077,6.487985e-18,1.350697e-17,9.021963e-16,8.957747e-20,...,0.0,0.0,0.0,0.0,0.756695,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,9.439474e-02,2655.202867,0.002811,3.681248e-04,1.989026e-03,0.013378,1.258885e-02,2.082301e-01,4.905254e-06,3.823138e-07,...,0.0,0.0,0.0,0.0,0.753251,0.0,0.0,0.0,0.0,0.0
496,9.576589e-02,2655.202867,0.002811,3.681248e-04,1.989026e-03,0.013378,1.258885e-02,2.082301e-01,4.905254e-06,3.823138e-07,...,0.0,0.0,0.0,0.0,0.753251,0.0,0.0,0.0,0.0,0.0
497,9.715696e-02,2655.202867,0.002811,3.681248e-04,1.989026e-03,0.013378,1.258885e-02,2.082301e-01,4.905254e-06,3.823138e-07,...,0.0,0.0,0.0,0.0,0.753251,0.0,0.0,0.0,0.0,0.0
498,9.856823e-02,2655.202867,0.002811,3.681248e-04,1.989026e-03,0.013378,1.258885e-02,2.082301e-01,4.905254e-06,3.823138e-07,...,0.0,0.0,0.0,0.0,0.753251,0.0,0.0,0.0,0.0,0.0


In [7]:
import cantera as ct

gas     = ct.Solution('gri30.yaml')

In [9]:
gas.TP  = 1000, 101325

In [11]:
gas.Y   = np.ones(len(gas.Y))

In [12]:
gas.Y

array([0.01886792, 0.01886792, 0.01886792, 0.01886792, 0.01886792,
       0.01886792, 0.01886792, 0.01886792, 0.01886792, 0.01886792,
       0.01886792, 0.01886792, 0.01886792, 0.01886792, 0.01886792,
       0.01886792, 0.01886792, 0.01886792, 0.01886792, 0.01886792,
       0.01886792, 0.01886792, 0.01886792, 0.01886792, 0.01886792,
       0.01886792, 0.01886792, 0.01886792, 0.01886792, 0.01886792,
       0.01886792, 0.01886792, 0.01886792, 0.01886792, 0.01886792,
       0.01886792, 0.01886792, 0.01886792, 0.01886792, 0.01886792,
       0.01886792, 0.01886792, 0.01886792, 0.01886792, 0.01886792,
       0.01886792, 0.01886792, 0.01886792, 0.01886792, 0.01886792,
       0.01886792, 0.01886792, 0.01886792])